In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import sklearn.metrics as metrics

In [2]:
# ---------- HyperParameters -----------
BATCH_SIZE = 32
lr = 0.001
epochs_n = 5


In [3]:
# transformations
transform = transforms.Compose([transforms.ToTensor()])


In [4]:
# download the training and validation dataset
trainset = torchvision.datasets.MNIST(root='data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True,num_workers=4)

testset = torchvision.datasets.MNIST(root='data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True,num_workers=4)


In [5]:
len(trainset), len(testset)

(60000, 10000)

In [9]:
# (((W - K + 2P)/S) + 1)
#     Here W = Input size
#     K = Filter size
#     S = Stride
#     P = Padding 

class MNISTModel(nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()

        # 28x28x1 => 26x26x32
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.d1 = nn.Linear(26 * 26 * 32, 128)
        self.d2 = nn.Linear(128, 10)

    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        x = self.conv1(x)
        x = F.relu(x)

        # flatten (view) => 32 x (32*26*26)
        x = x.flatten(start_dim=1)

        # 32 x (32*26*26) => 32x128
        x = self.d1(x)
        x = F.relu(x)

        #logits => 32x10
        logits = self.d2(x)
        out = F.softmax(logits, dim=1)
        return out
    
    

In [10]:
# Train the model, print out training accuracy
model = MNISTModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)



In [11]:
for epoch in range(epochs_n):
    train_running_loss = 0.0
    train_acc = 0.0

    # training step
    for i, (images, labels) in enumerate(trainloader):
        
        # forward + backprop + loss
        logits = model(images)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()

        # update model params
        optimizer.step()

        train_running_loss += loss.detach().item()
        train_acc += (torch.argmax(logits, 1).flatten() == labels).type(torch.float).mean().item()
    print(f'Epoch: {epoch:3d} | {train_running_loss / i:4f} | {train_acc /i:4f}')

Epoch:   0 | 1.631143 | 0.832761
Epoch:   1 | 1.498620 | 0.965965
Epoch:   2 | 1.481564 | 0.982140
Epoch:   3 | 1.477096 | 0.986526
Epoch:   4 | 1.473597 | 0.989478


In [12]:
test_acc = 0.0
for i, (images, labels) in enumerate(testloader, 0):
    outputs = model(images)
    print(f'Labels: {labels}')
    preds = torch.argmax(outputs, 1).flatten()
    print(f'Preds : {preds}')
    test_acc += (preds == labels).type(torch.float).mean().item()
    pscore = metrics.precision_score(preds.numpy(), labels.numpy(), average='micro')
    rscore = metrics.recall_score(preds.numpy(),labels.numpy(), average='micro')
    print(f'Precision Score:{pscore}, Recall Score:{rscore}')
print(f'Test Accuracy: {test_acc /i :.4f}')

Labels: tensor([8, 5, 7, 4, 5, 6, 5, 3, 4, 0, 1, 4, 0, 1, 8, 7, 2, 1, 6, 7, 4, 2, 4, 6,
        2, 1, 8, 0, 9, 4, 0, 0])
Preds : tensor([8, 5, 7, 4, 5, 6, 5, 3, 4, 0, 1, 4, 0, 1, 8, 7, 2, 1, 6, 7, 4, 2, 4, 6,
        2, 1, 8, 0, 9, 4, 0, 0])
Precision Score:1.0, Recall Score:1.0
Labels: tensor([8, 8, 3, 2, 8, 3, 7, 3, 4, 3, 0, 1, 9, 4, 1, 6, 7, 7, 9, 7, 6, 2, 1, 8,
        5, 5, 5, 9, 2, 4, 8, 1])
Preds : tensor([8, 8, 3, 2, 8, 3, 7, 3, 4, 3, 0, 1, 9, 4, 1, 6, 7, 7, 9, 7, 6, 2, 1, 8,
        5, 5, 5, 9, 2, 4, 8, 1])
Precision Score:1.0, Recall Score:1.0
Labels: tensor([2, 6, 9, 3, 7, 2, 6, 3, 3, 0, 1, 5, 9, 2, 9, 9, 3, 2, 7, 7, 2, 8, 9, 1,
        9, 4, 0, 6, 5, 6, 7, 5])
Preds : tensor([2, 6, 9, 3, 7, 2, 6, 3, 3, 0, 1, 5, 9, 8, 9, 9, 3, 2, 7, 7, 2, 8, 9, 1,
        9, 4, 8, 6, 5, 6, 7, 5])
Precision Score:0.9375, Recall Score:0.9375
Labels: tensor([1, 8, 8, 6, 6, 3, 3, 6, 7, 1, 1, 3, 9, 8, 5, 7, 8, 3, 9, 3, 8, 2, 1, 5,
        1, 5, 0, 6, 7, 4, 8, 2])
Preds : tensor([1, 8, 8, 6, 6, 3,